# Featuer Engineering

## Part 1: Data Merging WITH preserving NaNs

In [ ]:
# define root directory
import warnings
import re
import pandas as pd
import datetime


root_directory = 'C:\\Users/manue\\switchdrive\\Mutual Funds Project\\data\\predictors\\raw'

In [ ]:
import glob
sub_directories = glob.glob("C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\predictors\\raw\\*")
sub_directories[1]

In [ ]:
# get path to each file in each subdirectory
filepath = []
from os import walk
for folder in sub_directories:
    files = next(walk(folder), (None, None, []))[2]  # [] if no file
    for file in files:
        filepath.append(folder + '\\' + file)

In [ ]:
import calendar

# get all monthly dates
years = range(1980,2023,1)
months = range(1,13,1)

last_day_of_month = []
for year in years:
    for month in months:
        first, last = calendar.monthrange(year, month)
        last_day_of_month.append(datetime.datetime(year, month, last).date())
        

In [ ]:
#funds_IDs = pd.read_excel('C:/Users/manue/switchdrive/Mutual Funds Project/data/investment_list.xlsx')

warnings.filterwarnings('ignore')

regex = re.compile('(?s:.*){}(.*){}'.format(re.escape('\\'), re.escape('.csv')))

funds_IDs = list(pd.read_excel('C:/Users/manue/switchdrive/Mutual Funds Project/data/investment_list.xlsx')[0])

for date in last_day_of_month:
    print(date)
    string_date_format1 = date.strftime('%d/%m/%Y')
    string_date_format2 = date.strftime('%Y-%m-%d')
    monthly_df = pd.DataFrame(index= funds_IDs)
    for file in filepath:
        print(file)
        var_name = regex.findall(file)[0]
        
        variable_file = pd.read_csv(file, index_col = 'SecId')
        variable_file = variable_file.loc[variable_file.index.dropna()] #drop rows with no Fund ID
        variable_file = variable_file[~variable_file.index.duplicated(keep='first')] #drop duplicate entries
        if len(variable_file.columns) > 50: #we ignore data that are only available on a yearly basis
            try:
                variable_file = pd.DataFrame(variable_file.loc[:,string_date_format1])
                variable_file.rename(columns={string_date_format1: var_name}, inplace = True)
                monthly_df = pd.concat([monthly_df, variable_file], axis=1)

            except KeyError:
                variable_file = pd.DataFrame(variable_file.loc[:,string_date_format2])
                variable_file.rename(columns={string_date_format2: var_name}, inplace = True)
                monthly_df = pd.concat([monthly_df, variable_file], axis=1)
    monthly_df.dropna(how='all', inplace = True)

    monthly_df.to_csv('C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\features_new\\' + string_date_format2 + ".csv", header=True, index=True,)

        


## Part 2: Merging all months to single dataset

In [1]:
from os import walk
import pandas as pd

datasets_path = 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\features_new\\'
single_datasets = next(walk(datasets_path), (None, None, []))[2] 
#all_CSVs = [pd.read_csv(datasets_path+str(file)) for file in single_datasets]

In [6]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

subset1 = single_datasets[0:300]
subset2 = single_datasets[300:380]
subset3 = single_datasets[380:450]
subset4 = single_datasets[450:]
subsets = [subset1, subset2, subset3, subset4]

for i in range(0,4):
    subset = subsets[i]
    
    #read all CSVs
    all_CSVs_subset = [pd.read_csv(datasets_path+str(file)) for file in subset]
    df_sub = pd.concat(all_CSVs_subset)
    
    #drop artificial index and fundIDs
    df_sub = df_sub.iloc[:,2:]
    
    #converting data types to less memory intensive formats
    temp = pd.concat([
        df_sub.select_dtypes([], ['float64']),
        df_sub.select_dtypes(['float64']).apply(pd.Series.astype, dtype='float16')
        ], axis=1)

    temp = pd.concat([
            temp.select_dtypes([], ['object']),
            temp.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
            ], axis=1)
    
    #all floats are converted to float16 except the ones below where we use float 32 to prevent 'infs'
    temp.Rating = temp.Rating.astype('category')
    temp.returns = df_sub.returns.astype('float32')
    temp.Eq_Market_Value = df_sub.Eq_Market_Value.astype('float32') 
    temp.Total_Market_Value_Long = df_sub.Total_Market_Value_Long.astype('float32')
    temp.Total_Market_Value_Net = df_sub.Total_Market_Value_Net.astype('float32')
    temp.Total_Market_Value_Short  = df_sub.Total_Market_Value_Short.astype('float32')
    temp.Estimated_FundLevel_Net_Flow_Monthly_Comprehensive = df_sub.Estimated_FundLevel_Net_Flow_Monthly_Comprehensive.astype('float32')
    temp.Fund_Size_Monthly_Comprehensive = df_sub.Fund_Size_Monthly_Comprehensive.astype('float32')
    temp['Redemptions_FundLevel_N-PORT'] = df_sub['Redemptions_FundLevel_N-PORT'].astype('float32')
    temp['Reinvested_Dividends_FundLevel_N-PORT'] = df_sub['Reinvested_Dividends_FundLevel_N-PORT'].astype('float32')
    temp['Sales_FundLevel_N-PORT'] = df_sub['Sales_FundLevel_N-PORT'].astype('float32')
    temp.Info_Ratio_arith_1y  = df_sub.Info_Ratio_arith_1y.astype('float32')
    temp.Info_Ratio_arith_2y = df_sub.Info_Ratio_arith_2y.astype('float32')
    temp.Avg_Mkt_Cap_Mil_Long = df_sub.Avg_Mkt_Cap_Mil_Long.astype('float32')
    

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    newdf = temp.select_dtypes(include=numerics)
    print(newdf.columns.to_series()[np.isinf(newdf).any()]) # safety check to ensure no floats to 'infs' converted

    # compare how much memory is saved
    print('BEFORE CONVERTING')
    print(df_sub.info())
    print('AFTER CONVERTING')
    print(temp.info())
    
    #save to pickle file
    temp.to_pickle(f'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\pickle_files\\test_{i+1}.pkl') 
     

Series([], dtype: object)
BEFORE CONVERTING
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1075749 entries, 0 to 10651
Columns: 352 entries, #_Stock_Holdings_Long to Portfolio_Corporate_Sustainability_Score
dtypes: float64(348), object(4)
memory usage: 2.8+ GB
None
AFTER CONVERTING
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1075749 entries, 0 to 10651
Columns: 352 entries, #_Stock_Holdings_Long to Eq_Stylebox_Long
dtypes: category(5), float16(334), float32(13)
memory usage: 752.0 MB
None
Sortino_Ratio_1y    Sortino_Ratio_1y
dtype: object
BEFORE CONVERTING
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1019768 entries, 0 to 13067
Columns: 352 entries, #_Stock_Holdings_Long to Portfolio_Corporate_Sustainability_Score
dtypes: float64(348), object(4)
memory usage: 2.7+ GB
None
AFTER CONVERTING
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1019768 entries, 0 to 13067
Columns: 352 entries, #_Stock_Holdings_Long to Eq_Stylebox_Long
dtypes: category(5), float16(334), float32(1

In [13]:
DFs = [pd.read_pickle(f"C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\pickle_files\\test_{1+i}.pkl") for i in range(0,4)]
merged = pd.concat(DFs)
merged.Financial_Health_Grade_Long = merged.Financial_Health_Grade_Long.astype('category')
merged.Growth_Grade_Long = merged.Growth_Grade_Long.astype('category')
merged.Profitability_Grade_Long = merged.Profitability_Grade_Long.astype('category')
merged.to_pickle('C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\pickle_files\\full_dataset.pkl')




In [14]:
list(merged.select_dtypes(include=['object']).columns)

[]

In [17]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3737295 entries, 0 to 10848
Columns: 352 entries, #_Stock_Holdings_Long to Eq_Stylebox_Long
dtypes: category(5), float16(334), float32(13)
memory usage: 2.6 GB


In [20]:
data = pd.read_pickle('C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\pickle_files\\full_dataset.pkl')
sample = data.sample(n=100, random_state=1)


In [21]:
sample

,#_Stock_Holdings_Long,#_Stock_Holdings_Short,%_Asset_In_Top_10_Holdings,Eq_Market_Value,Total_Market_Value_Long,Total_Market_Value_Net,Total_Market_Value_Short,Cash_%_Net,Estimated_FundLevel_Net_Flow_Monthly_Comprehensive,Fund_Size_Monthly_Comprehensive,...,Eq_Type_Slow_Growth_Net,Eq_Type_Speculative_Growth_Net,Rating,Active_Share_ETF_Benchmark_Proxy,returns,Portfolio_Corporate_Sustainability_Score,Financial_Health_Grade_Long,Growth_Grade_Long,Profitability_Grade_Long,Eq_Stylebox_Long
10008,27.0,0.0,71.06250,130564920.0,168446752.0,167850656.0,5.961032e+05,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,0.0542,NaN,NaN,NaN,NaN,Mid Blend
1805,56.0,0.0,67.50000,183701376.0,184675520.0,184492000.0,1.835210e+05,0.000000,-2.705367e+06,1.845648e+08,...,2.660156,2.730469,1.0,NaN,-0.0150,49.125000,B-,C+,C+,Mid Blend
4412,36.0,NaN,37.78125,89996176.0,109145520.0,109145520.0,NaN,5.210938,1.678173e+05,1.091455e+08,...,NaN,NaN,3.0,89.6875,0.0107,20.343750,B+,B-,B,Large Growth
9476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,0.0564,NaN,NaN,NaN,NaN,NaN
8995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.111296e+04,1.402061e+07,...,NaN,NaN,2.0,NaN,-0.0384,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,35.0,0.0,57.65625,43796984.0,57411600.0,57298584.0,1.130164e+05,0.979980,-9.464938e+05,5.729858e+07,...,NaN,NaN,3.0,NaN,0.0259,NaN,NaN,C,C,Large Value
6728,58.0,3.0,33.03125,239514864.0,342354016.0,320186656.0,2.216738e+07,6.199219,-2.897609e+07,3.202660e+08,...,NaN,NaN,4.0,NaN,-0.0053,NaN,C-,C,C-,Large Blend
7296,55.0,NaN,42.81250,47263592.0,55548676.0,55548676.0,NaN,NaN,9.584769e+05,5.735520e+07,...,NaN,NaN,2.0,NaN,0.0038,24.453125,D,C+,C,Small Blend
7440,52.0,NaN,65.87500,471354624.0,538531776.0,538531776.0,NaN,1.610352,-3.923830e+06,5.385318e+08,...,NaN,NaN,2.0,NaN,-0.0202,23.718750,NaN,C,C,Large Blend


In [3]:
from sklearn.utils.fixes import loguniform
print(loguniform(1e-5, 100))